In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_459015/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [5]:
analize = Analizer(0.7)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_2_0,0.740071,0.049578,0.605674,0.225341,0.638837,0.132911,0.564212,0.173919,0.055665,0.121362,0.208805,0.364569,1.289921,0.377365,114.036152,191.102008,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
1,model_1_2_3,0.735661,0.007432,0.516127,0.148853,0.566270,0.135166,0.589231,0.213414,0.061161,0.145746,0.211081,0.367649,1.294839,0.380553,114.002504,191.068360,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
2,model_1_2_1,0.735530,0.034228,0.551937,0.211239,0.598312,0.135233,0.573324,0.197620,0.056678,0.134979,0.211366,0.367740,1.294986,0.380647,114.001513,191.067369,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
3,model_1_2_2,0.735459,0.021750,0.530927,0.186696,0.580659,0.135269,0.580731,0.206887,0.058442,0.140911,0.211432,0.367790,1.295065,0.380699,114.000973,191.066829,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
4,model_1_2_4,0.734040,-0.014539,0.495629,0.075961,0.544395,0.135995,0.602274,0.222455,0.066399,0.153097,0.211121,0.368775,1.296647,0.381718,113.990279,191.056135,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,model_1_2_8,0.576867,-0.111126,-0.426843,-2.332737,-0.357190,0.216363,0.659612,0.629315,0.239480,0.456056,0.241869,0.465149,1.471956,0.481475,113.061594,190.127450,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
71,model_1_2_9,0.576496,-0.044823,-0.429211,-2.727718,-0.396457,0.216553,0.620252,0.630360,0.267862,0.469251,0.234708,0.465352,1.472370,0.481685,113.059841,190.125697,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
72,model_1_2_10,0.574894,0.009890,-0.406132,-3.222816,-0.426680,0.217372,0.587772,0.620180,0.303439,0.479407,0.224369,0.466232,1.474157,0.482596,113.052290,190.118146,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
73,model_1_2_13,0.573470,0.075108,-0.411433,-4.418151,-0.544151,0.218100,0.549056,0.622519,0.389332,0.518880,0.205644,0.467012,1.475745,0.483403,113.045602,190.111458,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
